<a href="https://colab.research.google.com/github/JaySakarvadia/stock-lstm/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Features**
1. Close
2. Volume
3. HLC/3

**Labels**
1. Close



In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
GetAppleInformation = yf.Ticker("AAPL")
# print(GetAppleInformation.history(period="max"))
pd.set_option('display.max_rows', None)
stock_dataset = pd.DataFrame(GetAppleInformation.history(period="max"))
stock_dataset.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099192,0.099623,0.099192,0.099192,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094448,0.094448,0.094017,0.094017,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.087548,0.087548,0.087116,0.087116,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089273,0.089704,0.089273,0.089273,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.091861,0.092292,0.091861,0.091861,73449600,0.0,0.0


In [ ]:
stock_dataset["HLC/3"] = (stock_dataset["High"]+stock_dataset["Low"]+stock_dataset["Close"])/3
stock_dataset.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,HLC/3
Date,,,,,,,,
1980-12-12 00:00:00-05:00,0.099192,0.099623,0.099192,0.099192,469033600,0.0,0.0,0.099336
1980-12-15 00:00:00-05:00,0.094448,0.094448,0.094017,0.094017,175884800,0.0,0.0,0.094161
1980-12-16 00:00:00-05:00,0.087548,0.087548,0.087116,0.087116,105728000,0.0,0.0,0.087260
1980-12-17 00:00:00-05:00,0.089273,0.089704,0.089273,0.089273,86441600,0.0,0.0,0.089416
1980-12-18 00:00:00-05:00,0.091861,0.092292,0.091861,0.091861,73449600,0.0,0.0,0.092005


In [ ]:
training_set = pd.DataFrame(stock_dataset.loc[:, ['Close', 'Volume', 'HLC/3',]])
#training_set = pd.DataFrame(stock_dataset.loc[:, ['Close', 'Volume', 'HLC/3',]].values)

#print(training_set.shape)
training_set.head()
#training_set.info()

,Close,Volume,HLC/3
Date,,,
1980-12-12 00:00:00-05:00,0.099192,469033600,0.099336
1980-12-15 00:00:00-05:00,0.094017,175884800,0.094161
1980-12-16 00:00:00-05:00,0.087116,105728000,0.087260
1980-12-17 00:00:00-05:00,0.089273,86441600,0.089416
1980-12-18 00:00:00-05:00,0.091861,73449600,0.092005


In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = scaler.fit_transform(training_set)
print(training_set_scaled.shape)

(10889, 3)


In [ ]:
isinstance(training_set_scaled, np.ndarray) #training_set_scaled in a numpyarray

True

In [ ]:
features = []
labels = []
for i in range(60, len(training_set_scaled)):
    features.append(training_set_scaled[i-60:i, :]) #grabbing 60 days worth of data (60 day window) prior to i; grabing 60 rows before
    labels.append(training_set_scaled[i, 0])


features = np.array(features)
labels = np.array(labels)

features = np.reshape(features, (features.shape[0], features.shape[1], 3)) #redundant line? idk

In [ ]:
print(labels)
labels.shape

[1.85310256e-04 2.00570006e-04 1.96210115e-04 ... 9.17614164e-01
 9.21456099e-01 9.26056191e-01]


(10829,)

In [ ]:
print(features)
features.shape

[[[3.09576918e-04 6.31981003e-02 3.09806552e-04]
  [2.83417159e-04 2.36989104e-02 2.83627389e-04]
  [2.48533400e-04 1.42459064e-02 2.48717745e-04]
  ...
  [2.55073443e-04 1.56342786e-03 2.55989844e-04]
  [2.20193789e-04 2.06444914e-03 2.20357114e-04]
  [2.00570006e-04 3.82406004e-03 2.00718774e-04]]

 [[2.83417159e-04 2.36989104e-02 2.83627389e-04]
  [2.48533400e-04 1.42459064e-02 2.48717745e-04]
  [2.59433447e-04 1.16472357e-02 2.59625887e-04]
  ...
  [2.20193789e-04 2.06444914e-03 2.20357114e-04]
  [2.00570006e-04 3.82406004e-03 2.00718774e-04]
  [1.85310256e-04 4.02326127e-03 1.86174910e-04]]

 [[2.48533400e-04 1.42459064e-02 2.48717745e-04]
  [2.59433447e-04 1.16472357e-02 2.59625887e-04]
  [2.72517074e-04 9.89667945e-03 2.72719209e-04]
  ...
  [2.00570006e-04 3.82406004e-03 2.00718774e-04]
  [1.85310256e-04 4.02326127e-03 1.86174910e-04]
  [2.00570006e-04 7.98314033e-03 2.00718774e-04]]

 ...

 [[9.62286444e-01 5.13828425e-03 9.62965830e-01]
  [9.65669040e-01 5.33813224e-03 9.6830

(10829, 60, 3)

In [ ]:
print(features.shape)

(10829, 60, 3)


These dimensions can be explained as:


*   x = total # of sequences (for each index we get a sequence length of 60)
*   y = total length of each sequence
* z = total # of features in each sequence





In [ ]:
print(labels.shape)

(10829,)


In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (features.shape[1], 2)),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.LSTM(units = 50, return_sequences = True),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.LSTM(units = 50, return_sequences = True),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.LSTM(units = 50),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(units = 1)
# ])


In [ ]:
# model = Sequential()
# model.add(LSTM(units = 50, return_sequences = True, input_shape = (features.shape[1], 3)))
# model.add(Dropout(0.2))
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# model.add(LSTM(units = 50))
# model.add(Dropout(0.2))
# model.add(Dense(units = 1))

# #epochs = 20, batch_size = 32

In [ ]:
model = Sequential()
model.add(LSTM(60, return_sequences=True, input_shape=(features.shape[1], 3)))
model.add(Dropout(0.3))
model.add(LSTM(120, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Dense(1))

#epochs = 80, batch_size = 8


In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 60)            15360     
                                                                 
 dropout (Dropout)           (None, 60, 60)            0         
                                                                 
 lstm_1 (LSTM)               (None, 120)               86880     
                                                                 
 dropout_1 (Dropout)         (None, 120)               0         
                                                                 
 dense (Dense)               (None, 20)                2420      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 104681 (408.91 KB)
Trainable params: 10468

In [ ]:
loss = tf.keras.losses.MeanSquaredError()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #0.001 is default for adam

model.compile(optimizer=optimizer, loss=loss)

# model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# from time import time
# start = time()
# history = model.fit(features, labels, batch_size=8, epochs=20, validation_split=0.05, verbose=1)
# end = time()

from time import time
start = time()
history = model.fit(features, labels, batch_size=32, epochs=20, verbose=1)
end = time()

Epoch 1/20
339/339 [==============================] - 14s 8ms/step - loss: 0.0013
Epoch 2/20
339/339 [==============================] - 3s 8ms/step - loss: 5.8023e-04
Epoch 3/20
339/339 [==============================] - 3s 10ms/step - loss: 4.9917e-04
Epoch 4/20
339/339 [==============================] - 3s 8ms/step - loss: 5.0117e-04
Epoch 5/20
184/339 [===============>..............] - ETA: 1s - loss: 4.6428e-04

KeyboardInterrupt: 

In [ ]:
print('Total training time {} minutes'.format((end - start)/60))

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Loss Function')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Loss', 'Validation'], loc='upper right')
plt.show()